In [ ]:
import numpy as np
from keras.models import Model
from keras.applications import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

from keras.applications import imagenet_utils
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

In [ ]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
n_classes = len(np.unique(y_train))
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

In [ ]:
xception_model = Xception(weights='imagenet', include_top=False)

In [ ]:
x = xception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
out = Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs=xception_model.input, outputs=out)

In [ ]:
for layer in xception_model.layers:
    layer.trainable = False

In [ ]:
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=0)]


In [ ]:
n_epochs = 100
batch_size = 512
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.2, verbose=1, callbacks=callbacks)

In [ ]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

In [ ]:
for layer in model.layers[:115]:
    layer.trainable = False
for layer in model.layers[115:]:
    layer.trainable = True

In [ ]:
opt_finetune = Adam()
model.compile(optimizer=opt_finetune, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history_finetune = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.2, verbose=1, callbacks=callbacks)